In [7]:
#==================================================================
#   Brian's version 2.0
#
#   Use Chema's Tree Scheme
#   
#   10th Jan 2016 
#   Designed for MACS0647, image resolution 2048*2048, cosmic weight (z_fid = 3.0, z_lens = 0.584)
#------------------------------------------------------------------ 
#   

import pyfits
import numpy as np
from scipy import interpolate
import matplotlib.pyplot as plt
import math
from __future__ import division

mode = 0 # 0: separate all images, 1: stack all images
resolve = 1  #0: point source image, 1: resolved image
radius = 0.4 # size of the point source
Pix_ratio = 3173304.711
Npix = 2048
Npix0 = 512
ver = 'ver7'
delens_data_dir = '/Volumes/BRIAN/Research/relens/MACS0647/grid_error/' + ver + '/real_z/delens_data.dat' 
alphaX_512_rad_dir = '/Volumes/BRIAN/Research/WSLAP_results/grid_error/' + ver + '/reconstruction/recomp_alpha_x_rad_ver30.fits'
alphaY_512_rad_dir = '/Volumes/BRIAN/Research/WSLAP_results/grid_error/' + ver + '/reconstruction/recomp_alpha_y_rad_ver30.fits'
#alphaX_512_rad_dir = '/Volumes/BRIAN/Research/WSLAP_works/data/M0647/ver23/reconstruction/recomp_alpha_x_rad_ver23.fits'
#alphaY_512_rad_dir = '/Volumes/BRIAN/Research/WSLAP_works/data/M0647/ver23/reconstruction/recomp_alpha_y_rad_ver23.fits'
Img_dir = '/Volumes/BRIAN/Research/CLASH/MACS0647/cropped_images/macs0647_cropped_jpeg_r.fits'
outfile_dir = '/Volumes/BRIAN/Research/relens/MACS0647/grid_error/' + ver + '/real_z/'

#cosmic_weight parameters
z_f = float(3.0)   #redshift of the fiducial field
z2 = float(0.584)    #redshift of the lens

Omega = float(0.3)
Lambda = float(0.7)
Omega_k = float(1.0 - Omega - Lambda)
Dh = float(3.0e3)

###read the alphax file
hdulist = pyfits.open(alphaX_512_rad_dir)
alphaX_fid_512_rad = (hdulist[0].data)

###read the alphaY file
hdulist = pyfits.open(alphaY_512_rad_dir)
alphaY_fid_512_rad = (hdulist[0].data)

###read the image file
hdulist = pyfits.open(Img_dir)
Img = hdulist[0].data



#================================================================== 

# = * = * = * = * = * = * = * = * = * = * = * = * = * = * = * 
# 
#   Read the delens data file
#   
# = * = * = * = * = * = * = * = * = * = * = * = * = * = * = * 

f = file(delens_data_dir)
next(f) #skip the first line (header line)
parms = np.empty(6)
for line in f:
    line = line.strip()  #remove the \n at the end of line
    T = line.split("\t")  #split according to tab
    T = np.asarray(T)     #change list to array
    parms = np.vstack((parms, T))
parms = np.delete(parms, (0), axis=0)  #delete the first row (0,0,0...)
objID = parms[:,-1]  #extract the object ID column
parms = np.delete(parms, (-1), axis = 1) # delete the object ID column
parms = parms.astype(np.float)  #change from string format to float format
print('The  objects in the file are: ' + str(objID))
#print(parms)
print('The data file has dimension: ' + str(parms.shape))
x1_ALL = parms[:,0]
x2_ALL = parms[:,1]
y1_ALL = parms[:,2]
y2_ALL = parms[:,3]
#Ds_weight_ALL = parms[:,4]
redshift_ALL = parms[:,4]
# = * = * = * = * = * = * = * = * = * = * = * = * = * = * = * 
# 
#   Calculate the cosmic_weight for the sources
#   
# = * = * = * = * = * = * = * = * = * = * = * = * = * = * = * 
def LOS(z_input):
    z_end = float(z_input)
    
    z_step = int(1000000)
    z_start = float(5.0e-3)
    dz = float(z_end)/float(z_step)
    
    E_z_int = float(0.0)
    
    for iz in range(z_step+1):
        z = float(z_start + iz*dz)
        E_z_aux = float(math.sqrt(Omega*((1.0+z)**3) + Omega_k*((1.0+z)**2) + Lambda))
        E_z_int = E_z_int + float(dz/E_z_aux)
    
    Dm_output = float(Dh*E_z_int)
    
    return Dm_output

def cosmic_weight(z1):
    Dm_fid = LOS(z_f)
    Da_fid = Dm_fid/(1.0+z_f)

    Dm_source = LOS(z1)
    Da_source = Dm_source/(1.0+z1)

    Dm_lens = LOS(z2)
    Da_lens = Dm_lens/(1.0+z2)

    SQRT_lens = math.sqrt(1.0 + Omega_k*((Dm_lens**2)/(Dh**2)))
    SQRT_fid = math.sqrt(1.0 + Omega_k*((Dm_fid**2)/(Dh**2)))
    SQRT_source = math.sqrt(1.0 + Omega_k*((Dm_source**2)/(Dh**2)))

    Da_fid_lens=(1.0/(1.0+z_f))*((Dm_fid*SQRT_lens)-(Dm_lens*SQRT_fid))

    Aux1 = Dm_source*SQRT_lens-Dm_lens*SQRT_source
    Da_source_lens=(1.0/(1.0+z1))*(Aux1)

    Aux1 = Da_fid_lens*Da_source
    weight_source=float(Da_fid*Da_source_lens)/float(Aux1)

    print'Redshift:   ' + str(z1)
    print'*** Cosmic Weight *** :   ' + str(weight_source)
    
    return weight_source
    
Ds_weight_ALL = [cosmic_weight(num) for num in redshift_ALL]

print('Ds_weight_ALL' + str(Ds_weight_ALL))



    

# = * = * = * = * = * = * = * = * = * = * = * = * = * = * = * 
# 
#   Create point source image file for resolve = 0
#   
# = * = * = * = * = * = * = * = * = * = * = * = * = * = * = * 
if resolve == 0:
    Img = np.zeros((Npix,Npix))
    for i in np.arange(x1_ALL.shape[0]-1):
        xAvg = int((x1_ALL[i] + x2_ALL[i])/2)
        yAvg = int((y1_ALL[i] + y2_ALL[i])/2)
        objID = objID.astype(np.float)
        Img[(yAvg - radius):(yAvg + radius),(xAvg - radius):(xAvg + radius)] = objID[i]
        #x1_ALL[i] = int(xAvg - radius)
        #x2_ALL[i] = int(xAvg + radius)
        #y1_ALL[i] = int(yAvg - radius)
        #y2_ALL[i] = int(yAvg + radius)


# = * = * = * = * = * = * = * = * = * = * = * = * = * = * = * 
# 
# Interpolate alpha
#   
# = * = * = * = * = * = * = * = * = * = * = * = * = * = * = * 
print(' Interpolating alpha ... ')
#alpha_X
x = np.arange(Npix0)
y = np.arange(Npix0)
fX = interpolate.interp2d(x, y, alphaX_fid_512_rad)

ratio = (Npix0)/Npix
xnew = np.arange(0,Npix0,ratio)
ynew = np.arange(0,Npix0,ratio)
alphaX_fid_ACS_rad = fX(xnew, ynew)

print('alphaX has dimension:' + str(alphaX_fid_ACS_rad.shape))

#alpha_Y
fY = interpolate.interp2d(x, y, alphaY_fid_512_rad)
alphaY_fid_ACS_rad = fY(xnew, ynew)

print('alphaY has dimension:' + str(alphaY_fid_ACS_rad.shape))

print('Interpolation Done')

alphaX_fid_ACS = alphaX_fid_ACS_rad*Pix_ratio
alphaY_fid_ACS = alphaY_fid_ACS_rad*Pix_ratio

# = * = * = * = * = * = * = * = * = * = * = * = * = * = * = * 
# 
# Define a function that does delens and relens
#   
# = * = * = * = * = * = * = * = * = * = * = * = * = * = * = * 

def delensrelens(x1,x2,y1,y2,Ds_weight):
    x1 = int(x1)
    x2 = int(x2)
    y1 = int(y1)
    y2 = int(y2)
    alphaX_ACS = alphaX_fid_ACS * Ds_weight
    alphaY_ACS = alphaY_fid_ACS * Ds_weight
    #Find the delensed area
    XMIN = Npix
    XMAX = 0
    YMIN = Npix
    YMAX = 0

    for i in np.arange(x1,x2):
        for j in np.arange(y1,y2):
            BetaX_ACS = i - alphaX_ACS[j,i]
            BetaY_ACS = j - alphaY_ACS[j,i]
        
            if (BetaX_ACS < XMIN):
                XMIN = BetaX_ACS
            if (BetaX_ACS > XMAX):
                XMAX = BetaX_ACS
            if (BetaY_ACS < YMIN):
                YMIN = BetaY_ACS
            if (BetaY_ACS > YMAX):
                YMAX = BetaY_ACS

    #add a small buffer
    XMIN = int(XMIN)-13
    XMAX = int(XMAX)+13
    YMIN = int(YMIN)-13
    YMAX = int(YMAX)+13
    print('XMIN, XMAX: ' + str(XMIN),str(XMAX))
    print('YMIN, YMAX: ' + str(YMIN),str(YMAX))
# = * = * = * = * = * = * = * = * = * = * = * = * = * = * = * 
# 
#use a tree to store the positions
#with a box size Nxy=Nx*Ny=(XMAX-XMIN)*(YMAX-YMIN)
#use an index system based on this box where for a given
#position x,y in the ACS image the index based in this
#box is Index_box = (y-YMIN)*Nx + (x-XMIN)
#store in an array of size
#Array_Index_box_CC = LONGARR(Nxy,50) the CC rays that
#fall in a given ACS pixel, x,y with index Index_box.
#

    Nx = XMAX-XMIN
    Ny = YMAX-YMIN
    Npix_box = Nx * Ny
    MaxNhits = 500
    Array_Indx_box_CC = np.zeros((MaxNhits,Npix_box))
    ThetaX_ACS = np.zeros(53770)
    ThetaY_ACS = np.zeros(53770)
    BetaX_ACS_Rg1 = np.zeros(53770)
    BetaY_ACS_Rg1 = np.zeros(53770)

    print('Start delensing, & store delensed positions in TREE ... ')

    CC = -1

    for i in np.arange(x1,x2):
        for j in np.arange(y1,y2):
            CC = CC + 1
            ThetaX_ACS[CC] = i
            ThetaY_ACS[CC] = j
            BetaX_ACS_Rg1[CC] = i - alphaX_ACS[j,i]
            BetaY_ACS_Rg1[CC] = j - alphaY_ACS[j,i]
            ii = int(BetaX_ACS_Rg1[CC])
            jj = int(BetaY_ACS_Rg1[CC])
        
            #Store in tree
            Indx_box = (jj - YMIN) * Nx + (ii - XMIN)
            Array_Indx_box_CC[0,Indx_box] = Array_Indx_box_CC[0,Indx_box] + 1
            if (Array_Indx_box_CC[0,Indx_box] > MaxNhits - 1):
                Array_Indx_box_CC[0,Indx_box] = MaxNhits - 1
            kkk = int(Array_Indx_box_CC[0,Indx_box])
            Array_Indx_box_CC[kkk,Indx_box] = CC

    print(' Delens & TREE storing done ')
# = * = * = * = * = * = * = * = * = * = * = * = * = * = * = * 
# 
#   
#   
# = * = * = * = * = * = * = * = * = * = * = * = * = * = * = * 
#Relens
    print(' Relensing ... ')

    ReImg = np.zeros((Npix,Npix))

    for i in np.arange(2048):
        for j in np.arange(2048):
            BetaX_ACS = i - alphaX_ACS[j,i]
            BetaY_ACS = j - alphaY_ACS[j,i]
            if (BetaX_ACS > XMIN and BetaX_ACS < XMAX-1\
            and BetaY_ACS > YMIN and BetaY_ACS < YMAX-1):
                ii = int(BetaX_ACS)
                jj = int(BetaY_ACS)
                Indx_box = int((jj - YMIN) * Nx + (ii - XMIN))
                N_hits = int(Array_Indx_box_CC[0,Indx_box])
                MinDist = 100.0
                CC_ok = int(Array_Indx_box_CC[1,Indx_box])
                for kk in np.arange(N_hits):
                    CC = int(Array_Indx_box_CC[kk,Indx_box])
                    dBx = BetaX_ACS_Rg1[CC] - BetaX_ACS
                    dBy = BetaY_ACS_Rg1[CC] - BetaY_ACS
                    BetaDist = math.sqrt(dBx ** 2 + dBy ** 2)
                    if (BetaDist < MinDist):
                        CC_ok = CC
                        MinDist = BetaDist
                ReImg[j,i] = Img[int(ThetaY_ACS[CC_ok]),int(ThetaX_ACS[CC_ok])]

    print(' Relens done, start printing the relensed image ... ')

#    hdu = pyfits.PrimaryHDU(ReImg)
#    hdulist = pyfits.HDUList([hdu])
#    hdulist.writeto(outfile)

#    print(' Relensed Image Printed in File ')
    return ReImg

if mode == 0:
    for i in np.arange(parms.shape[0]):
        ReImg = delensrelens(x1_ALL[i],x2_ALL[i],y1_ALL[i],y2_ALL[i],Ds_weight_ALL[i])
        hdu = pyfits.PrimaryHDU(ReImg)
        hdulist = pyfits.HDUList([hdu])
        hdulist.writeto(outfile_dir + 'relens_' + str(objID[i]) + '.fits')
        print(' Relensed Image ' + str(objID[i]) + ' Printed in File ')
if mode == 1:
    for i in np.arange(parms.shape[0]):
        ReImg = delensrelens(x1_ALL[i],x2_ALL[i],y1_ALL[i],y2_ALL[i],Ds_weight_ALL[i])
        if i == 0:
            ReImgStack = ReImg
        else:
            ReImgStack = ReImgStack + ReImg
        print(' Image ' + str(objID[i]) + ' Relensed ')

    hdu = pyfits.PrimaryHDU(ReImgStack)
    hdulist = pyfits.HDUList([hdu])
    hdulist.writeto(outfile_dir + 'relens_ALL.fits')
    print(' Relensed Image (All) Printed In File ')



The  objects in the file are: ['3.1' '3.2' '6.1' '6.2' '6.3']
The data file has dimension: (5, 5)
Redshift:   3.1
*** Cosmic Weight *** :   1.00751491302
Redshift:   3.1
*** Cosmic Weight *** :   1.00751491302
Redshift:   11.0
*** Cosmic Weight *** :   1.17517077802
Redshift:   11.0
*** Cosmic Weight *** :   1.17517077802
Redshift:   11.0
*** Cosmic Weight *** :   1.17517077802
Ds_weight_ALL[1.0075149130221903, 1.0075149130221903, 1.1751707780183207, 1.1751707780183207, 1.1751707780183207]
 Interpolating alpha ... 
alphaX has dimension:(2048, 2048)
alphaY has dimension:(2048, 2048)
Interpolation Done
('XMIN, XMAX: 912', '944')
('YMIN, YMAX: 1057', '1086')
Start delensing, & store delensed positions in TREE ... 
 Delens & TREE storing done 
 Relensing ... 
 Relens done, start printing the relensed image ... 
 Relensed Image 3.1 Printed in File 
('XMIN, XMAX: 915', '947')
('YMIN, YMAX: 1058', '1089')
Start delensing, & store delensed positions in TREE ... 
 Delens & TREE storing done 
 R

In [2]:
mode = 1 # 0: separate all images, 1: add all images
f = file('delens_data.dat')
next(f) #skip the first line (header line)
parms = np.empty(6)
for line in f:
    line = line.strip()  #remove the \n at the end of line
    T = line.split("\t")  #split according to tab
    T = np.asarray(T)     #change list to array
    parms = np.vstack((parms, T))
parms = np.delete(parms, (0), axis=0)  #delete the first row (0,0,0...)
objID = parms[:,-1]  #extract the object ID column
parms = np.delete(parms, (-1), axis = 1) # delete the object ID column
parms = parms.astype(np.float)  #change from string format to float format
print('The  objects in the file are: ' + str(objID))
#print(parms)
print('The data file has dimension: ' + str(parms.shape))
x1_ALL = parms[:,0]
x2_ALL = parms[:,1]
y1_ALL = parms[:,2]
y2_ALL = parms[:,3]
Ds_weight_ALL = parms[:,4]

def test(x1,x2,y1,y2,Ds_weight):
    return(x1+x2+y1+y2+Ds_weight)

if mode == 0:
    for i in np.arange(parms.shape[0]):
        ReImg = test(x1_ALL[i],x2_ALL[i],y1_ALL[i],y2_ALL[i],Ds_weight_ALL[i])
        print(ReImg)
if mode == 1:
    for i in np.arange(parms.shape[0]):
        ReImg = test(x1_ALL[i],x2_ALL[i],y1_ALL[i],y2_ALL[i],Ds_weight_ALL[i])
        if i == 0:
            ReImgStack = ReImg
        else:
            ReImgStack = ReImgStack + ReImg
            print(ReImgStack)

resolve = 0  #0: point source image, 1: resolved image
print(x1_ALL.shape)
if resolve == 0:
    Img = np.zeros((Npix,Npix))
    for i in np.arange(x1_ALL.shape[0]-1):
        xAvg = (x1_ALL[i] + x2_ALL[i])/2
        yAvg = (y1_ALL[i] + y2_ALL[i])/2
        objID = objID.astype(np.float)
        Img[yAvg,xAvg] = objID[i]




The  objects in the file are: ['1.1' '1.2' '1.3' '2.1' '2.2' '2.3' '3.1' '3.2' '4.1' '4.2' '4.3' '5.1'
 '5.2' '6.1' '6.2' '6.3' '7.1' '7.2' '7.3' '8.1' '8.2' '9.1' '9.2' '9.3']
The data file has dimension: (24, 5)
8086.4
11982.6
14694.3
17261.0
19043.7
22008.8
24963.9
30300.8
35237.7
41071.6
46940.1
51844.6
54515.6
57193.6
61818.6
66584.8
70105.0
74697.2
79717.5
84857.8
90137.7
94091.6
99699.5
(24,)


/Users/chanmingyan/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:47: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


[ 12.  13.  14.  15.  16.  17.  18.  19.  20.  21.  22.  23.]
